In [1]:
# 필요한 모듈 import

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# 데이터 파일 업로드

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sub = pd.read_csv('./sample_submission.csv')

In [4]:
# EDA 및 데이터 전처리

In [5]:
train.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


In [6]:
test.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt
0,1,9,1800,Gd,2,702,Ex,1800,1800,Ex,2,2007,2007,2007
1,2,6,1082,TA,1,240,TA,1082,1082,TA,1,1948,1950,1948
2,3,6,1573,Gd,2,440,Gd,756,769,Gd,2,2000,2000,2000
3,4,6,2443,Gd,3,744,Gd,1158,1158,Gd,2,2004,2004,2004
4,5,5,1040,TA,2,686,TA,1040,1040,TA,1,1968,1968,1991


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 15 columns):
id                1350 non-null int64
Overall Qual      1350 non-null int64
Gr Liv Area       1350 non-null int64
Exter Qual        1350 non-null object
Garage Cars       1350 non-null int64
Garage Area       1350 non-null int64
Kitchen Qual      1350 non-null object
Total Bsmt SF     1350 non-null int64
1st Flr SF        1350 non-null int64
Bsmt Qual         1350 non-null object
Full Bath         1350 non-null int64
Year Built        1350 non-null int64
Year Remod/Add    1350 non-null int64
Garage Yr Blt     1350 non-null int64
target            1350 non-null int64
dtypes: int64(12), object(3)
memory usage: 158.3+ KB


In [8]:
test.info() # train, test 모두 결측치 없음, train과 test 비율 1:1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 14 columns):
id                1350 non-null int64
Overall Qual      1350 non-null int64
Gr Liv Area       1350 non-null int64
Exter Qual        1350 non-null object
Garage Cars       1350 non-null int64
Garage Area       1350 non-null int64
Kitchen Qual      1350 non-null object
Total Bsmt SF     1350 non-null int64
1st Flr SF        1350 non-null int64
Bsmt Qual         1350 non-null object
Full Bath         1350 non-null int64
Year Built        1350 non-null int64
Year Remod/Add    1350 non-null int64
Garage Yr Blt     1350 non-null int64
dtypes: int64(11), object(3)
memory usage: 147.8+ KB


In [30]:
X_t.isnull().sum()

Overall Qual       0
Gr Liv Area        0
Garage Cars        0
Garage Area        0
Total Bsmt SF      0
1st Flr SF         0
Full Bath          0
Year Built         0
Year Remod/Add     0
Garage Yr Blt      0
Exter Qual_Ex      0
Exter Qual_Fa      0
Exter Qual_Gd      0
Exter Qual_TA      0
Kitchen Qual_Ex    0
Kitchen Qual_Fa    0
Kitchen Qual_Gd    0
Kitchen Qual_Po    0
Kitchen Qual_TA    0
Bsmt Qual_Ex       0
Bsmt Qual_Fa       0
Bsmt Qual_Gd       0
Bsmt Qual_Po       0
Bsmt Qual_TA       0
dtype: int64

In [8]:
# 품질 부분 encoding
# 나머지 scaling 후 회귀분석

In [10]:
# encoding을 위해 품질 부분 고유값 확인

print(train['Exter Qual'].unique(),train['Kitchen Qual'].unique(),train['Bsmt Qual'].unique())
print(test['Exter Qual'].unique(),test['Kitchen Qual'].unique(),test['Bsmt Qual'].unique())
# train과 test의 kichen qual 부분 고유값이 다름 -> label encoding, 나머지는 one hot encoding

['Ex' 'Gd' 'TA' 'Fa'] ['Ex' 'Gd' 'TA' 'Fa'] ['Ex' 'TA' 'Gd' 'Fa' 'Po']
['Gd' 'TA' 'Ex' 'Fa'] ['Ex' 'TA' 'Gd' 'Fa' 'Po'] ['Ex' 'TA' 'Gd' 'Fa' 'Po']


In [13]:
# label encoding (for문 이용)

train_kichen = []
test_kichen = []

# train data
for i in range(len(train)):
    if train['Kitchen Qual'][i]=='Ex':
        train_kichen.append(0)
    elif train['Kitchen Qual'][i]=='Gd':
        train_kichen.append(1)
    elif train['Kitchen Qual'][i]=='TA':
        train_kichen.append(2)
    elif train['Kitchen Qual'][i]=='Fa':
        train_kichen.append(3)
    else:
        train_kichen.append(4)

# test data
for i in range(len(train)):
    if test['Kitchen Qual'][i]=='Ex':
        test_kichen.append(0)
    elif test['Kitchen Qual'][i]=='Gd':
        test_kichen.append(1)
    elif test['Kitchen Qual'][i]=='TA':
        test_kichen.append(2)
    elif test['Kitchen Qual'][i]=='Fa':
        test_kichen.append(3)
    else:
        test_kichen.append(4)
        
train['Kitchen Qual'] = train_kichen
test['Kitchen Qual'] = test_kichen

In [14]:
# One Hot encoding

train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [15]:
# X, y 정의
X = train.drop(['id', 'target'],axis=1)
y = train.target
X_t = test.drop(['id'], axis=1)

In [16]:
# scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_t_scaled = scaler.transform(X_t)

In [17]:
# 학습, 검증 데이터 분리

X_train, X_val, y_train, y_val = train_test_split(X_scaled,y,test_size=0.3,random_state=42)

In [18]:
# SGD reg 사용
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor()
sgd.fit(X_train, y_train)
y_val_pred = sgd.predict(X_val)

In [19]:
# 평가식 정의
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [20]:
# 모델 학습 검증
NMAE(y_val, y_val_pred)

0.10656182796714901

In [22]:
# 제출 파일 생성
y_pred = sgd.predict(X_t_scaled)
sub['target'] = y_pred
sub.to_csv('./sample_submission.csv', index=False)

In [33]:
# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

params = {'penalty':['l2', 'l1', 'elasticnet'], 'loss':['squared_error', 'huber','epsilon_insensitive','squared_epsilon_insensitive'],
         'alpha':np.arange(0,0.01,0.00005)}
grid = GridSearchCV(sgd, param_grid=params, n_jobs=-1,scoring=mean_absolute_error,cv=10, verbose=1)
grid.fit(X_train, y_train)

Fitting 10 folds for each of 2400 candidates, totalling 24000 fits


C:\Users\박세진\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  category=UserWarning,


GridSearchCV(cv=10, estimator=SGDRegressor(), n_jobs=-1,
             param_grid={'alpha': array([0.00e+00, 5.00e-05, 1.00e-04, 1.50e-04, 2.00e-04, 2.50e-04,
       3.00e-04, 3.50e-04, 4.00e-04, 4.50e-04, 5.00e-04, 5.50e-04,
       6.00e-04, 6.50e-04, 7.00e-04, 7.50e-04, 8.00e-04, 8.50e-04,
       9.00e-04, 9.50e-04, 1.00e-03, 1.05e-03, 1.10e-03, 1.15e-03,
       1.20e-03, 1.25e-03, 1.30e-03, 1.35e-03, 1.40e-03, 1.45e-03...
       9.00e-03, 9.05e-03, 9.10e-03, 9.15e-03, 9.20e-03, 9.25e-03,
       9.30e-03, 9.35e-03, 9.40e-03, 9.45e-03, 9.50e-03, 9.55e-03,
       9.60e-03, 9.65e-03, 9.70e-03, 9.75e-03, 9.80e-03, 9.85e-03,
       9.90e-03, 9.95e-03]),
                         'loss': ['squared_error', 'huber',
                                  'epsilon_insensitive',
                                  'squared_epsilon_insensitive'],
                         'penalty': ['l2', 'l1', 'elasticnet']},
             scoring=<function mean_absolute_error at 0x000002ADFC1D2E58>,
             verbos

In [34]:
grid.best_params_ # default값...

{'alpha': 0.0, 'loss': 'squared_error', 'penalty': 'l2'}

In [37]:
# 새로운 모델 생성 및 학습

sgd2 = SGDRegressor(alpha=0) # alpha를 아예 0으로 줌(default=0.0001)
sgd2.fit(X_train, y_train)
y_val_pred = sgd2.predict(X_val)

NMAE(y_val, y_val_pred)

0.10648460462217416

In [39]:
# 제출 파일 생성
y_pred = sgd2.predict(X_t_scaled)
sub['target'] = y_pred
sub.to_csv('./sample_submission2.csv', index=False) # 제출 점수: NMAE 0.1140391906

In [41]:
# 다른 모델 선택
# DT
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
y_val_pred = dt.predict(X_val)

NMAE(y_val, y_val_pred)

0.1323365796662401

In [42]:
# RF
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_val_pred = rf.predict(X_val)

NMAE(y_val, y_val_pred) # 오 성능 좋은데

0.09915169282356424

In [44]:
# 제출 파일 생성 - RF
y_pred = rf.predict(X_t_scaled)
sub['target'] = y_pred
sub.to_csv('./sample_submission3.csv', index=False) # 제출 점수: NMAE 0.1053759727